In [28]:
import os, json, gc
import pandas as pd
import numpy as np
from scipy.spatial import distance
from scipy.stats import pearsonr
from urllib2 import Request, urlopen
from pandas.io.json import json_normalize


BASE_URI = os.environ.get('API_URI') or "http://0.0.0.0:3000/"
SOCIAL_INTERESTS_URI = BASE_URI + "api/v4/re/interests/social"
BUSINESS_INTERESTS_URI = BASE_URI + "api/v4/re/interests/business"
LIFESTYLE_INTERESTS_URI = BASE_URI + "api/v4/re/interests/lifestyle"
EVENT_TYPES_URI = BASE_URI + "api/v4/re/events/types"
CONNECTIONS_URI = BASE_URI + "api/v4/re/connections"
ACCOUNTS_URI = BASE_URI + "api/v4/re/accounts"

def func(uri):
    df_accounts = _request_data(ACCOUNTS_URI)
    df_accounts['location'].fillna('empty', inplace=True)

    df_profile = _request_data(uri)
    
    for index, account in df_accounts.iterrows():
        print type(account.location)
        if not df_profile[df_profile.account_id == account.id].empty:
            _interests_sim_with_loc(account, df_profile)

def _index_of(id, df):
    print id
    return df[df.account_id == id].index.tolist()[0]
            
def _request_data(uri):
    print ("Sending request to:", uri)
    request=Request(uri)
    data = json.loads(urlopen(request).read())
    df = pd.DataFrame(data)
    print ("Data shape:", df.shape)
    return df

def _interests_sim_with_loc(account, df_profile):
    # df_new = _matrix_with_loc(location)
    # df_profile_t = df_new[df_new.columns[14:]]
    df_new = _interests_matrix_with_loc(account.location, df_profile)
    df_profile_t = df_new[df_new.columns[2:]]

    index = _index_of(account.id, df_new)

    profile_d = distance.pdist(df_profile_t, metric='matching')
    profile_D = distance.squareform(profile_d)
    p_r = profile_D[index].tolist()
    del profile_d
    del profile_D
    gc.collect()

    sim = pd.Series(p_r)
    df_new['interest_similarity'] = 1 - sim.values
    df_new['interest_count'] = (1 - sim.values) * len(df_profile_t.columns)
    df_new['interest_count'] = df_new['interest_count'].astype(int)
    
    # List = [i[0] for i in sorted(enumerate(p_r), key=lambda x:x[1])][0:21]
    # return df_new.ix[List][['account_id', 'interest_count']]#['V2']
    return df_new


def _interests_matrix_with_loc(location, df):
    df_profile = df.copy()
    df_profile[['account_id']] = df_profile[['account_id']].astype(int)
    df_profile['location'].fillna('empty', inplace=True)
    df_profile_t = pd.pivot_table(df_profile, index=['account_id', 'location'], columns=['social'], values='indicator')
    del df_profile
    gc.collect()

    df_profile_t = df_profile_t.reset_index()
    df_profile_t = df_profile_t.fillna(value=0)
    df_profile_t = df_profile_t[(df_profile_t.location == '') | (df_profile_t.location == 'empty') | (df_profile_t.location.str.contains(location))]
    df_profile_f = df_profile_t.dropna()
    del df_profile_t
    gc.collect()

    df_profile_s = df_profile_f.sort_values(by='account_id', ascending=True)
    del df_profile_f
    gc.collect()

    df_profile_s.reset_index(drop=True, inplace=True)
    return df_profile_s



In [29]:
func("http://0.0.0.0:3000/api/v4/re/interests/social")

('Sending request to:', 'http://0.0.0.0:3000/api/v4/re/accounts')
('Data shape:', (37313, 3))
                               email     id                    location
0            dwurry@post.harvard.edu    428           San Francisco, CA
1                adixit@bubblews.com  20670                       empty
2                 cnimro@hotmail.com   3834                     Toronto
3                sargam486@gmail.com  28058                       empty
4               jordanbudi@gmail.com  43509              Miami, Florida
5                 itsdante@gmail.com  29354                       empty
6                roybcohen@gmail.com  45485                       empty
7                dcgerritz@gmail.com  28075                       empty
8           anthony1.small@yahoo.com  46669                       empty
9                 smumanac@gmail.com  17153                  Boston, MA
10             ruby.k.bola@gmail.com  11074                    New York
11     elizabeth.j.scherle@gmail.com     8

KeyboardInterrupt: 